In [10]:
###################importing libraries##############################
from __future__ import print_function
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from openpyxl import load_workbook

In [11]:
####################Reading the csv Files####################################
x_train=pd.read_excel(r'class_project\B_train_samples.xlsx')
y_train=pd.read_excel(r'class_project\B_train_labels.xlsx')
x_test=pd.read_excel(r'class_project\B_test_samples.xlsx')
y_test=pd.read_excel(r'class_project\B_test_labels.xlsx')
x_train=x_train[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22']]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
type(y_train)

(96756, 22)
(96756, 25)
(48377, 22)
(48377, 1)


pandas.core.frame.DataFrame

In [30]:
####################scaling the training Data######################################
scaler = MinMaxScaler()
x_train_scl = scaler.fit_transform(x_train)
x_test_scl = scaler.transform(x_test)
#x_train_scl=to_dataframe(x_train_scl)
x_train = pd.DataFrame(data=x_train_scl)
x_test=pd.DataFrame(data=x_test_scl)

In [31]:
################Creating Function to fit and evaluate the data###################
def fit_and_evaluate(t_x, val_x, t_y, val_y, EPOCHS=100, BATCH_SIZE=1000):
    results = model.fit(t_x, t_y, epochs=EPOCHS, batch_size=BATCH_SIZE, 
              verbose=1, validation_split=0.1)  
    print("Val Score: ", model.evaluate(val_x, val_y))
    return results

In [14]:
y_train=y_train[['B','C5','C10', 'C15', 'C20', 'C25', 'C30', 'C35', 'C40', 'A5', 'A10', 'A15', 'A20', 'A25', 'A30', 'A35', 'A40', 'N5', 'N10', 'N15', 'N20', 'N25', 'N30', 'N35', 'N40']]

In [1]:
#############Creating loop for each column in y_train#################
from pandas import ExcelWriter
accuracy = []
f_score = []
y_pred_list = []

from keras.layers import GaussianNoise
t=y_train.columns
count=0
for i in range(y_train.shape[1]):
    temp = ""
    temp1 = ""
    y_train_1=y_train[t[i]]
    y_train_1=pd.DataFrame(y_train_1)
    model = Sequential()
    # in the first layer, you must specify the expected input data shape:
    # here, 43-dimensional vectors.
    model.add(Dense(64, activation='relu', input_dim=22))

    model.add(Dense(128, activation='relu'))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    model.add(Dense(11, activation='softmax'))
    
    model.add(Dense(512, activation = 'linear'))

    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    n_folds=10
    #save the model history in a list after fitting so that we can plot later
    model_history = [] 

    for i in range(10):
        print("Training on Fold: ",i+1)
        t_x, val_x, t_y, val_y = train_test_split(x_train, y_train_1, test_size=0.1,
                                                  random_state = np.random.randint(1,1000, 1)[0])
        model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y, 2,1000))
        score = model.evaluate(x_test,y_test,batch_size=1000)
        temp += str(score[1]) + ","
        y_pred=model.predict_classes(x_test, verbose=0)
        
        f1 = f1_score(y_test, y_pred, average="micro")
        temp1 += str(f1) + ","
    y_pred_list.append(y_pred)
    
    
    accuracy.append(temp)
    f_score.append(temp1)
    
df11 = pd.DataFrame({'B': y_pred_list[0], 'C5': y_pred_list[1], 'C10': y_pred_list[2], 'C15': y_pred_list[3], 'C20': y_pred_list[4], 'C25': y_pred_list[5], 'C30': y_pred_list[6], 'C35': y_pred_list[7], 'C40': y_pred_list[8], 'A5': y_pred_list[9], 'A10': y_pred_list[10], 'A15': y_pred_list[11], 'A20': y_pred_list[12], 'A25': y_pred_list[13], 'A30': y_pred_list[14], 'A35': y_pred_list[15], 'A40': y_pred_list[16], 'N5': y_pred_list[17], 'N10': y_pred_list[18], 'N15': y_pred_list[19], 'N20': y_pred_list[20], 'N25': y_pred_list[21], 'N30': y_pred_list[22], 'N35': y_pred_list[23], 'N40': y_pred_list[24]})
writer = ExcelWriter("b_prediction.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()

print("accuracy")
print(accuracy)
print("f-score")
print(f_score)

data_a = []
for i in range(25):
    data_a.append(list(map(float, accuracy[i].split(",")[:10])))
    
data_b = []

for i in range(len(data_a)):
    temp = data_a[i]
    temp.append(statistics.mean(temp))
    temp.append(statistics.stdev(temp))
    data_b.append(temp)
    
df11 = pd.DataFrame({'N1': data_b[0], 'N2': data_b[1], 'N3': data_b[2], 'N4': data_b[3], 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_b_a.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()


data_a = []
for i in range(25):
    data_a.append(list(map(float, f_score[i].split(",")[:10])))
    
data_b = []

for i in range(len(data_a)):
    temp = data_a[i]
    temp.append(statistics.mean(temp))
    temp.append(statistics.stdev(temp))
    data_b.append(temp)
    
df11 = pd.DataFrame({'N1': data_b[0], 'N2': data_b[1], 'N3': data_b[2], 'N4': data_b[3], 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_b_f.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()

Using TensorFlow backend.


NameError: name 'y_train' is not defined

In [15]:
print(len(val_x))

17535
